In [131]:
#import common libs
import numpy as np
import pandas as pd
import os
from datetime import datetime

#define CSV's paths
IN_PATH_CSV = "../data/entrada-produtos.csv"
OUT_PATH_CSV = "../data/saida-produtos.csv"
FINAL_PATH_CSV = "../data/saida-final.csv"

#date format
DATE_FORMAT = "%d/%m/%Y"

#read CSVs
ins = pd.read_csv(IN_PATH_CSV)
outs = pd.read_csv(OUT_PATH_CSV)

data_compra = None
data_venda = None
final = np.array([["codigo-fornecedor", "dias"]])

for index, id_fornecedor in enumerate(outs["codigo-fornecedor"]):    
    list_of_ins = ins[ins["codigo-fornecedor"] == id_fornecedor]
    if not list_of_ins.empty:
        codigo_fornecedor = list_of_ins.iloc[0]["codigo-fornecedor"]
        data_venda = datetime.strptime(outs.iloc[index]["data-saida"], DATE_FORMAT)
        data_compra = datetime.strptime(list_of_ins.iloc[0]["data-entrada"], DATE_FORMAT)
        diff = (data_venda - data_compra).days
        if diff > 0:
            final = np.append(final, [codigo_fornecedor,diff])
reshaped_matrix = final.reshape(-1,2) 
final_df = pd.DataFrame(reshaped_matrix)
final_df.to_csv(FINAL_PATH_CSV, index=False, header=False)

In [144]:
import pandas as pd
import numpy as np
from datetime import datetime

# Define data format
data_format = '%d/%m/%Y'

# Define file paths
input_file = "../data/entrada-produtos.csv"
output_file = "../data/saida-produtos.csv"
final_file = "../data/saida-final.csv"
merged_file = "../data/merged.csv"

# Read CSV files
input_df = pd.read_csv(input_file)
output_df = pd.read_csv(output_file)

# Convert date columns to datetime format
input_df['data-entrada'] = pd.to_datetime(input_df['data-entrada'], format=data_format)
output_df['data-saida'] = pd.to_datetime(output_df['data-saida'], format=data_format)

# Merge DataFrames based on 'codigo-fornecedor'
merged_df = output_df.merge(input_df, on='codigo-fornecedor')

# Agrupar por 'codigo-fornecedor' e ordenar por 'data-entrada'
merged_df = merged_df.sort_values(['codigo-fornecedor', 'data-entrada'])

# Calculate the difference in days
merged_df['dias'] = (merged_df['data-saida'] - merged_df['data-entrada']).dt.days

# Calcular a quantidade cumulativa para cada fornecedor e data de entrada
#merged_df['quantidade_acumulada'] = merged_df.groupby('codigo-fornecedor')['quantidade'].cumsum()

# Criar uma nova coluna para rastrear a quantidade restante em cada saída
merged_df['quantidade_restante'] = merged_df['quantidade']
merged_df.to_csv(merged_file, index=False)

# Função para atualizar a quantidade restante para cada saída
def update_quantidade_restante(row):
    row['quantidade_restante'] -= 1
    return row

# Aplicar a função a cada linha, resetando a quantidade restante para cada fornecedor
final_df = merged_df.groupby('codigo-fornecedor').apply(lambda x: x.apply(update_quantidade_restante, axis=1))

print(final_df.columns)
# Filtrar apenas as linhas com data de saída e quantidade restante positiva
final_df = final_df[final_df['data-saida'].notna() & final_df['quantidade_restante'] > 0]

# Filter for positive differences and select relevant columns
final_df = merged_df[merged_df['dias'] > 0][['codigo-fornecedor', 'dias', 'data-entrada','data-saida', 'descricao']]

# Save the final DataFrame to a CSV file
final_df.to_csv(final_file, index=False)

Index(['codigo-fornecedor', 'data-saida', 'data-entrada', 'descricao',
       'quantidade', 'dias', 'quantidade_acumulada', 'quantidade_restante'],
      dtype='object')


/var/folders/k3/rslfg54n4ng2797x4rrh_cn40000gn/T/ipykernel_43060/3833872325.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = merged_df.groupby('codigo-fornecedor').apply(lambda x: x.apply(update_quantidade_restante, axis=1))
